<a href="https://colab.research.google.com/github/mindswim/connect4-zero/blob/main/notebooks/train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect4-Zero Training on Colab

AlphaZero-style training for Connect 4. Make sure to use a GPU runtime:
- Runtime > Change runtime type > GPU (T4 is fine, A100 is faster)

In [ ]:
# Check GPU
!nvidia-smi

Tue Dec 30 16:05:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# Clone repo
!git clone https://github.com/mindswim/connect4-zero.git
%cd connect4-zero

Cloning into 'connect4-zero'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 53 (delta 3), reused 53 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (53/53), 41.55 KiB | 13.85 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/connect4-zero


In [ ]:
# Install dependencies
!pip install -e . -q

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for connect4-zero (pyproject.toml) ... done


In [ ]:
# Verify CUDA is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

CUDA available: True
Device: NVIDIA A100-SXM4-40GB


In [ ]:
# Quick benchmark
!c4z benchmark --sims 100 --games 5

Using device: cuda
Running 5 games with 100 simulations...
Game 1: 33 moves
Game 2: 38 moves
Game 3: 38 moves
Game 4: 41 moves
Game 5: 36 moves

Total time: 37.02s
Games/sec: 0.14
Moves/sec: 5.02
Sims/sec: 502


In [ ]:
# Train! This will take a while but show progress
# Use --iterations to control how long
!c4z train --iterations 50

Using device: cuda
               Configuration                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Parameter                  ┃ Value       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ mcts.num_simulations       │ 100         │
│ mcts.c_puct                │ 1.5         │
│ mcts.dirichlet_alpha       │ 0.3         │
│ mcts.dirichlet_epsilon     │ 0.25        │
│ mcts.temp_threshold        │ 10          │
│ network.num_channels       │ 64          │
│ network.num_blocks         │ 5           │
│ train.batch_size           │ 256         │
│ train.lr                   │ 0.001       │
│ train.weight_decay         │ 0.0001      │
│ train.train_steps_per_iter │ 300         │
│ train.value_loss_weight    │ 1.0         │
│ selfplay.games_per_iter    │ 25          │
│ selfplay.max_moves         │ 42          │
│ selfplay.augment_data      │ True        │
│ eval.num_matches           │ 100         │
│ eval.accept_threshold      │ 0.55        │
│ eval.eval_simulations      │ 100  

In [ ]:
# Save checkpoint to Google Drive (optional)
from google.colab import drive
drive.mount('/content/drive')

!cp -r checkpoints /content/drive/MyDrive/connect4-zero-checkpoints

In [ ]:
# Test against random player
from connect4zero.net import load_checkpoint
from connect4zero.eval import Arena
from connect4zero.utils import get_device

device = get_device()
model, _ = load_checkpoint('checkpoints/best.pt', device)

arena = Arena(num_simulations=100, device=device)
result = arena.evaluate_vs_random(model, num_games=50)
print(f"Win rate vs random: {result.win_rate*100:.1f}%")

In [ ]:
# Download best model
from google.colab import files
files.download('checkpoints/best.pt')